In [1]:
import torch
import torch.nn as nn
from utils import data_loader
from vae_classifier_attacks import test_attack, fgsm_attack
import pickle
from utils.viewer import show_batch
import matplotlib.pyplot as plt
from utils.mmd import MMD_torch

In [2]:
train_set, test_set = data_loader.get_data()
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

In [3]:
# alpha =1
# B=batch_size

# def MMD(x, y):
#     x = x.view(x.size(0), x.size(2) * x.size(3))
#     y = y.view(y.size(0), y.size(2) * y.size(3))

#     xx, yy, zz = torch.mm(x,x.t()), torch.mm(y,y.t()), torch.mm(x,y.t())

#     rx = (xx.diag().unsqueeze(0).expand_as(xx))
#     ry = (yy.diag().unsqueeze(0).expand_as(yy))

#     K = torch.exp(- alpha * (rx.t() + rx - 2*xx))
#     L = torch.exp(- alpha * (ry.t() + ry - 2*yy))
#     P = torch.exp(- alpha * (rx.t() + ry - 2*zz))

#     beta = (1./(B*(B-1)))
#     gamma = (2./(B*B)) 

#     return beta * (torch.sum(K)+torch.sum(L)) - gamma * torch.sum(P)

In [4]:
class NeuralModel(nn.Module):
    def __init__(self):
        super().__init__()

        num_channels = 8

        self.conv = nn.Sequential(

            nn.Conv2d(1, num_channels, kernel_size=5),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(num_channels, eps=1e-05, momentum=0.5, affine=True),
            nn.ReLU(inplace=True),

            nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_channels, eps=1e-05, momentum=0.5, affine=True),
            nn.ReLU(inplace=True),

            nn.Conv2d(num_channels, num_channels, kernel_size=5),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(num_channels, eps=1e-05, momentum=0.5, affine=True),
            nn.ReLU(inplace=True)

        )
        self.fc1 = nn.Linear(num_channels * 4 ** 2, 20)
        self.fc2 = nn.Linear(20, 10)

    def forward(self, x):
        convolved = self.conv(x)
        after_fc1 = self.fc1(convolved.view(convolved.size(0), -1))
        #output = self.fc2(after_fc1)
        output = after_fc1
        return output
    
    def encode(self, x):
        
        convolved = self.conv(x)
        code = self.fc1(convolved.view(convolved.size(0), -1))
        
        return code
         


In [5]:
def train_model(model, train_data):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    n_epochs = 10
    model.train()
    
    model.to(device)
    
    print("training ...")
    for epoch in range(n_epochs):

        for batch in train_data:
            batch_images, batch_labels = batch
            
            batch_images = batch_images.to(device)
            batch_labels = batch_labels.to(device)

            batch_output = model(batch_images)
            loss = criterion(batch_output, batch_labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    print("Done training..")
    print("=*="*20)
    return model



In [6]:
batch_size = 512
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

In [7]:
#model = NeuralModel()
#train_model(model, train_loader)
#test_model(model, test_loader)

In [8]:
eps = 0.3

In [9]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=1)
#acc, adv_examples = test_attack(model, device, train_loader, epsilon=eps)

In [10]:
#print(len(adv_examples))

In [11]:
# with open('models/train_adv_examples.pkl', 'wb') as f:
#     pickle.dump(adv_examples, f)

In [12]:
with open('models/train_adv_examples.pkl', 'rb') as f:
    adv_examples = pickle.load(f)

In [13]:
true_labels =[]
adv_labels = [] 
adv_images =[]

In [14]:
for example in adv_examples:
    true_l, adv_l, adv_img = example
    
    true_labels.append(true_l)
    adv_labels.append(adv_l)
    adv_images.append(adv_img)

In [15]:
true_labels = torch.Tensor(true_labels)
adv_images = torch.Tensor(adv_images).reshape(-1,1,28,28)

In [16]:
pure_images = train_set.data.reshape(-1,1,28,28)

b_size = 60
training_data = []
for b in range (0, pure_images.shape[0], b_size):
    
    batch_images = torch.cat((pure_images[b: b+b_size].float(),adv_images[b: b+b_size]),0)
    batch_flags = torch.cat((torch.zeros(len(pure_images[b: b+b_size])),torch.ones(len(pure_images[b: b+b_size]))),0) 
    batch_labels = torch.cat((true_labels[b: b+b_size], true_labels[b: b+b_size]),0)
    
    training_data.append((batch_images, batch_flags, batch_labels))

In [17]:
b1 = next(iter(training_data))

a,b,c = b1

print(len(a))
print(b)
print(c)

120
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([5., 0., 4., 1., 9., 2., 1., 3., 1., 4., 3., 5., 3., 6., 1., 7., 2., 8.,
        6., 9., 4., 0., 9., 1., 1., 2., 4., 3., 2., 7., 3., 8., 6., 9., 0., 5.,
        6., 0., 7., 6., 1., 8., 7., 9., 3., 9., 8., 5., 9., 3., 3., 0., 7., 4.,
        9., 8., 0., 9., 4., 1., 5., 0., 4., 1., 9., 2., 1., 3., 1., 4., 3., 5.,
        3., 6., 1., 7., 2., 8., 6., 9., 4., 0., 9., 1., 1., 2., 4., 3., 2., 7.,
        3., 8., 6., 9., 0., 5., 6., 0., 7., 6., 1., 8., 7.,

In [18]:
def train_MMD(model, train_data, alpha = 1.0):
    
    criterion = nn.CrossEntropyLoss()
    learning_rate = .001
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    n_epochs = 5
    
    model.train()
    model.to(device)
    
    print("training ...")
    for epoch in range(n_epochs):
        for batch in train_data:
            batch_images, batch_flags, batch_labels = batch    
            
            batch_images = batch_images.to(device)
            batch_flags = batch_flags.long()
            batch_flags = batch_flags.to(device)
            
            batch_labels = batch_labels.long()
            batch_labels = batch_labels.to(device)
            
            #show_batch(batch_images[0:5].cpu())
          
            batch_clean = batch_images[0:60].clone()
            batch_adv = batch_images[60:].clone()
            
            batch_labels_clean = batch_labels[0:60].clone()
            batch_output = model.fc2(model(batch_clean))
            
            #latent_code = model(batch_images.clone())
            
            down_stream_loss = criterion(batch_output, batch_labels_clean)
            #representation_loss = alpha * MMD_torch(latent_code.clone(), batch_flags.clone())
            
            #total_loss = down_stream_loss + representation_loss
            
            optimizer.zero_grad()
            down_stream_loss.backward()
            optimizer.step()
            
            print("the classification loss after processing this batch is: ", down_stream_loss.item())
            #print("The representation loss after processing this batch is: ", representation_loss.item())
            print("")
    print("Done training..")
    print("=*="*20)

    return model

In [ ]:
modelMM = NeuralModel()
modelMM = train_MMD(modelMM, training_data, alpha = .00)

training ...
the classification loss after processing this batch is:  2.3208706378936768

the classification loss after processing this batch is:  2.293311834335327

the classification loss after processing this batch is:  2.332824468612671

the classification loss after processing this batch is:  2.3023955821990967

the classification loss after processing this batch is:  2.2828381061553955

the classification loss after processing this batch is:  2.252190113067627

the classification loss after processing this batch is:  2.218926191329956

the classification loss after processing this batch is:  2.2116575241088867

the classification loss after processing this batch is:  2.250243663787842

the classification loss after processing this batch is:  2.2485005855560303

the classification loss after processing this batch is:  2.2185561656951904

the classification loss after processing this batch is:  2.1671528816223145

the classification loss after processing this batch is:  2.208796977

the classification loss after processing this batch is:  0.38961273431777954

the classification loss after processing this batch is:  0.3795948028564453

the classification loss after processing this batch is:  0.3646095097064972

the classification loss after processing this batch is:  0.4598725736141205

the classification loss after processing this batch is:  0.3425110876560211

the classification loss after processing this batch is:  0.43057969212532043

the classification loss after processing this batch is:  0.429582804441452

the classification loss after processing this batch is:  0.29550614953041077

the classification loss after processing this batch is:  0.3925382196903229

the classification loss after processing this batch is:  0.3214670419692993

the classification loss after processing this batch is:  0.2541467249393463

the classification loss after processing this batch is:  0.3033197522163391

the classification loss after processing this batch is:  0.228209525346755

the classification loss after processing this batch is:  0.1237567737698555

the classification loss after processing this batch is:  0.1176547259092331

the classification loss after processing this batch is:  0.09412410110235214

the classification loss after processing this batch is:  0.12375400960445404

the classification loss after processing this batch is:  0.24522456526756287

the classification loss after processing this batch is:  0.22387883067131042

the classification loss after processing this batch is:  0.1546202450990677

the classification loss after processing this batch is:  0.13044388592243195

the classification loss after processing this batch is:  0.1862824708223343

the classification loss after processing this batch is:  0.1693112850189209

the classification loss after processing this batch is:  0.10769940167665482

the classification loss after processing this batch is:  0.15961408615112305

the classification loss after processing this batch is:  0.1074040308

the classification loss after processing this batch is:  0.17028869688510895

the classification loss after processing this batch is:  0.19893456995487213

the classification loss after processing this batch is:  0.10098900645971298

the classification loss after processing this batch is:  0.08563721925020218

the classification loss after processing this batch is:  0.03759380057454109

the classification loss after processing this batch is:  0.13969646394252777

the classification loss after processing this batch is:  0.08141308277845383

the classification loss after processing this batch is:  0.09088286757469177

the classification loss after processing this batch is:  0.0975787416100502

the classification loss after processing this batch is:  0.19531574845314026

the classification loss after processing this batch is:  0.14707423746585846

the classification loss after processing this batch is:  0.11878896504640579

the classification loss after processing this batch is:  0.212411

the classification loss after processing this batch is:  0.09021254628896713

the classification loss after processing this batch is:  0.03461891785264015

the classification loss after processing this batch is:  0.06125052645802498

the classification loss after processing this batch is:  0.058028776198625565

the classification loss after processing this batch is:  0.04628118500113487

the classification loss after processing this batch is:  0.0423114039003849

the classification loss after processing this batch is:  0.05249513313174248

the classification loss after processing this batch is:  0.07285265624523163

the classification loss after processing this batch is:  0.14866240322589874

the classification loss after processing this batch is:  0.10313312709331512

the classification loss after processing this batch is:  0.10242360085248947

the classification loss after processing this batch is:  0.03136051446199417

the classification loss after processing this batch is:  0.03326

...

the classification loss after processing this batch is:  0.10006996989250183

the classification loss after processing this batch is:  0.07844887673854828

the classification loss after processing this batch is:  0.011567473411560059

the classification loss after processing this batch is:  0.016088517382740974

the classification loss after processing this batch is:  0.005945062730461359

the classification loss after processing this batch is:  0.021696725860238075

the classification loss after processing this batch is:  0.01214830856770277

the classification loss after processing this batch is:  0.10528063029050827

the classification loss after processing this batch is:  0.03841293603181839

the classification loss after processing this batch is:  0.04014177247881889

the classification loss after processing this batch is:  0.24778799712657928

the classification loss after processing this batch is:  0.01525411568582058

the classification loss after processing this batch is:  0.0

the classification loss after processing this batch is:  0.07450878620147705

the classification loss after processing this batch is:  0.011755100451409817

the classification loss after processing this batch is:  0.002188650658354163

the classification loss after processing this batch is:  0.023805102333426476

the classification loss after processing this batch is:  0.023890741169452667

the classification loss after processing this batch is:  0.03555057942867279

the classification loss after processing this batch is:  0.01711328886449337

the classification loss after processing this batch is:  0.06131928786635399

the classification loss after processing this batch is:  0.013980905525386333

the classification loss after processing this batch is:  0.029656855389475822

the classification loss after processing this batch is:  0.05108918249607086

the classification loss after processing this batch is:  0.07872965186834335

the classification loss after processing this batch is:  0

the classification loss after processing this batch is:  0.03811156749725342

the classification loss after processing this batch is:  0.027847401797771454

the classification loss after processing this batch is:  0.04814394190907478

the classification loss after processing this batch is:  0.0982799232006073

the classification loss after processing this batch is:  0.03417886048555374

the classification loss after processing this batch is:  0.009084701538085938

the classification loss after processing this batch is:  0.03838192671537399

the classification loss after processing this batch is:  0.02452688291668892

the classification loss after processing this batch is:  0.007297039031982422

the classification loss after processing this batch is:  0.010784021578729153

the classification loss after processing this batch is:  0.0025810240767896175

the classification loss after processing this batch is:  0.008839782327413559

the classification loss after processing this batch is:  0

the classification loss after processing this batch is:  0.13417914509773254

the classification loss after processing this batch is:  0.002355162287130952

the classification loss after processing this batch is:  0.011520052328705788

the classification loss after processing this batch is:  0.01039990596473217

the classification loss after processing this batch is:  0.01954917050898075

the classification loss after processing this batch is:  0.007496571633964777

the classification loss after processing this batch is:  0.006968124769628048

the classification loss after processing this batch is:  0.0320022888481617

the classification loss after processing this batch is:  0.05165921151638031

the classification loss after processing this batch is:  0.024320563301444054

the classification loss after processing this batch is:  0.05237273499369621

the classification loss after processing this batch is:  0.03708129748702049

the classification loss after processing this batch is:  0.0

the classification loss after processing this batch is:  0.02807067148387432

the classification loss after processing this batch is:  0.04894816875457764

the classification loss after processing this batch is:  0.02447887696325779

the classification loss after processing this batch is:  0.005787753965705633

the classification loss after processing this batch is:  0.006728227715939283

the classification loss after processing this batch is:  0.05777443200349808

the classification loss after processing this batch is:  0.02617206610739231

the classification loss after processing this batch is:  0.02943267859518528

the classification loss after processing this batch is:  0.009935100562870502

the classification loss after processing this batch is:  0.034542251378297806

the classification loss after processing this batch is:  0.009684745222330093

the classification loss after processing this batch is:  0.11867145448923111

the classification loss after processing this batch is:  0.

In [ ]:
def test_model(model, test_data):
    model.eval()
    model.to(device)
    correct = 0

    for batch in test_data:
        batch_images, batch_labels = batch

        batch_images = batch_images.to(device)
        batch_labels = batch_labels.to(device)

        predictions = model(batch_images)

        predictions = predictions.data.max(1, keepdim=True)[1]
        correct += predictions.eq(batch_labels.data.view_as(predictions)).sum()

    accuracy = float(correct.item() / len(test_data.dataset))
    
    print("The classifier accuracy is: ", 100 * accuracy)
    
    return accuracy


In [ ]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64)
test_model(modelMM, test_loader)

The classifier accuracy is:  9.82


0.0982

In [ ]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1)
acc, _ = test_attack(modelMM, device, test_loader, epsilon=eps)